# Vehicle Detection Project

The goals / steps of this project are the following:

* Define functions to compute input features from dataset car/non-car images.
    - Color space change, Spatial binning, Color histogram, HOG with subsampling
* Combine (concatenate) input features, normalize/scale (using sklearn StandardScaler) them.
* Using a balanced dataset and shuffled train/test splits, train a single or ensemble of classifiers to detect cars.
    - Linear SVM, Decision trees, Deep neural networks
* Implement a generic sliding window technique that feeds patches from a given image to classifier for classification.
    - Use multiple window sizes
    - Come up with efficient scan strategy (e.g. only bottom half of image)
* Implement heatmap thresholding to combine multiple detections and remove false positives
* Use information from multiple frames to remove spurious false positives that appear in one or small number of frames.
    - Calculate motion trajectory if required


In [1]:
import numpy as np
import cv2
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.preprocessing import StandardScaler
%matplotlib inline

/root/miniconda3/envs/carnd_term_1/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')
/root/miniconda3/envs/carnd_term_1/lib/python3.5/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
### Helper Functions

def plotImageFiles(filenames):
    for f in filenames:
        fig = plt.figure()
        fig.set_size_inches(3,6)
        img = mpimg.imread(f)
        plt.title(f)
        plt.axis('off')
        plt.imshow(img)
        
def plotImage(img, title=None, cmap=None):
    fig = plt.figure(figsize=(10, 12))
    #fig.set_size_inches(4,8)
    if title is not None:
        plt.title(title)
    plt.axis('off')
    if cmap is None:
        plt.imshow(img)
    else:
        plt.imshow(img, cmap=cmap)
    
def plotMultipleImages(images, labels=None, ptitle=None, cmap=None):
    """ This function will plot the images specified in a
    single plot.
    """
    numImages = len(images)
    ii = 1
    for img in images:
        fig = plt.figure()
        if labels is not None:
            plt.title(labels[ii-1], fontsize="xx-small")
        plt.axis('off')
        if cmap is not None:
            plt.imshow(img.squeeze(), cmap=cmap)
        else:
            plt.imshow(img.squeeze())
        ii += 1
        
def plotImageAndPolygon(img, polygonPts):
    fig, ax = plt.subplots(1)
    pgn = plt.Polygon(polygonPts, color='r', alpha=0.25, lw=2)
    ax.imshow(img)
    ax.add_patch(pgn)
    
def plotImageAndMultiplePolygons(img, multPolyPts1, multPolyPts2, cmap=None):
    fig, ax = plt.subplots(1)
    if cmap is None:
        ax.imshow(img)
    else:
        ax.imshow(img, cmap=cmap)
    for p in range(0, len(multPolyPts1)):
        pgn1 = plt.Polygon(multPolyPts1[p], color='r', alpha=0.25, lw=1)
        pgn2 = plt.Polygon(multPolyPts2[p], color='b', alpha=0.25, lw=1)
        ax.add_patch(pgn1)
        ax.add_patch(pgn2)
        
def plot3SideBySide(oimg, timg, hist):
    fig = plt.figure(figsize=(12,3))
    plt.subplot(131)
    plt.imshow(oimg)
    plt.subplot(132)
    plt.imshow(timg, cmap="gray")
    plt.subplot(133)
    plt.plot(hist)
    
def plotLaneFit(timg, llfit, rlfit):
    plt.figure()
    # Generate x and y values for plotting
    ploty = np.linspace(0, timg.shape[0]-1, timg.shape[0] )
    left_fitx = llfit[0]*ploty**2 + llfit[1]*ploty + llfit[2]
    right_fitx = rlfit[0]*ploty**2 + rlfit[1]*ploty + rlfit[2]
    plt.imshow(timg, cmap="gray")
    plt.plot(left_fitx, ploty, color='r', lw=3)
    plt.plot(right_fitx, ploty, color='b', lw=3)
    plt.xlim(0, 1280)
    plt.ylim(720, 0)
    
print("*** Helper Functions Defined ***")

*** Helper Functions Defined ***


### Compute Input Features

In [ ]:
def getBinSpatialFeatures(img, size=(32, 32)):
    """ Computes spatially binned color features
    """
    features = cv2.resize(img, size).ravel() 
    return features

def getColorHistogramFeatures(img, nbins=32, bins_range=(0, 256)):
    """ Computes color histogram of features for each image channel
    and combines them into a single feature vector.
    """
    channelhists = []
    for i in range(0, img.shape[2]):
        channelhist = np.histogram(img[:,:,i], bins=nbins, range=bins_range)
        channelhists.append(channelhist)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate(channelhists)
    return hist_features

def getHOGFeatures(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=True):
    """ Computes and returns HOG features and visualization (optional).
    """
    if vis == True:
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    else:      
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=True, 
                       visualise=vis, feature_vector=feature_vec)
        return features
    
def extractCombinedImageFeatures(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    """ Returns a combined image feature vector for given image.
    """
    #1) Define an empty list to receive features
    img_features = []
    #2) Apply color conversion if other than 'RGB'
    if color_space != 'RGB':
        if color_space == 'HSV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
        elif color_space == 'LUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
        elif color_space == 'HLS':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
        elif color_space == 'YUV':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        elif color_space == 'YCrCb':
            feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    else:
        feature_image = np.copy(img)      
    #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = getBinSpatialFeatures(feature_image, size=spatial_size)
        #4) Append features to list
        img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = getColorHistogramFeatures(feature_image, nbins=hist_bins)
        #6) Append features to list
        img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.extend(getHOGFeatures(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))      
        else:
            hog_features = getHOGFeatures(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
        img_features.append(hog_features)

    #9) Return concatenated array of features
    return np.concatenate(img_features)


print("*** Input Features Computation Functions Defined ***")

### Classifier Training

In [ ]:
print("*** Classifier Training Functions Defined ***")

### Sliding Window Technique

In [ ]:
    
print("*** Sliding Window Technique Functions Defined ***")


### Combining Overlaps and False Positives Removal

In [ ]:
print("*** Overlap Combining and False Positives Removal Functions Defined ***")


### Overall Pipeline

In [ ]:
def pipeline():
    pass

In [ ]:
def testPipelineOnTestImages():
    tfiles = glob.glob('test_images/test*.jpg')
    for tfile in tfiles:
        timg = mpimg.imread(tfile)
        plt.figure(figsize=(10, 12))
        plt.imshow(timg)

#testPipelineOnTestImages()


### Vehicle Detection on Video

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#TODO: Basic pre-processing

def process_image(image):
    return image

In [ ]:
prevllfit = None
prevrlfit = None
outvideofile = 'project_video_output.mp4'
pvideo = VideoFileClip("project_video.mp4")
ovideo = pvideo.fl_image(process_image)
%time ovideo.write_videofile(outvideofile, audio=False)

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(outvideofile))